In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.corpus import stopwords
from sklearn.metrics import pairwise_distances
from nltk.stem import wordnet
from nltk import word_tokenize
from nltk import pos_tag
from nltk.sentiment import SentimentIntensityAnalyzer
import random

In [2]:
df = pd.read_excel('C:/Users\Salim\Desktop\AI Huddersfield 2022\lecture notes\week 11\D5\dialog_talk_agent.xlsx')

In [3]:
df.head()

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,NaN
3,Describe yourself,NaN
4,tell me about yourself,NaN


In [4]:
df = df.ffill()
df.head()

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder


In [5]:
def text_nomalization(text):
    text = str(text).lower()
    clean_text = re.sub(r'[^ a-z0-9]', '', text)
    tokens = word_tokenize(clean_text)
    lemma = wordnet.WordNetLemmatizer()
    pos = pos_tag(tokens, tagset = None)
    words = []
    for token, tag in pos:
        if tag.startswith('V'):
            pos_val = 'v'
        elif tag.startswith('J'):
            pos_val = 'a'
        elif tag.startswith('R'):
            pos_val = 'r'
        else:
            pos_val = 'n'
        lemma_list = lemma.lemmatize(token, pos_val)
        words.append(lemma_list)
    return ' '.join(words)

In [6]:
print(text_nomalization("I'm telling you that you are annoying")) # testing the function

im tell you that you be annoy


In [7]:
df['lematized_context'] = df['Context'].apply(text_nomalization) #apply text_nomalization to the Context column
df.head()

,Context,Text Response,lematized_context
0,Tell me about your personality,Just think of me as the ace up your sleeve.,tell me about your personality
1,I want to know you better,I can help you work smarter instead of harder,i want to know you good
2,Define yourself,I can help you work smarter instead of harder,define yourself
3,Describe yourself,I can help you work smarter instead of harder,describe yourself
4,tell me about yourself,I can help you work smarter instead of harder,tell me about yourself


Now we are going to create our first chatbot function using Bag of Words (BOW)

In [8]:
def bow(text):
    cv = CountVectorizer()
    vectors = cv.fit_transform(df['lematized_context']).toarray() #creating an array of vectors from lematized column of df
    features = cv.get_feature_names_out() #getting all feature names of the vecotr array
    df_vectors = pd.DataFrame(vectors, columns = features) # store vector arrays in to pandas dataframe. see below
    #stop = stopwords.words('english')    
    Q_lema = text_nomalization(text) #lemmatize input sentense and return lemmatized sentense
    text_vector= cv.transform([Q_lema]).toarray()
    cosine = 1-pairwise_distances(df_vectors, text_vector, metric = 'cosine' )
    max_val_index = cosine.argmax() # returns index val with max cosine similarity
    return ( df['Text Response'].loc[max_val_index])

In [9]:
cv = CountVectorizer()
v = cv.fit_transform(df['lematized_context']).toarray()
f =cv.get_feature_names_out()
vdf = pd.DataFrame(v, columns = f)
vdf.head()

,21,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,...,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Our chatbot model is basically complete now we can create a UI for easy of chatting:

In [10]:
Responses = {'Quit' : ['No', 'nop', 'no thank you', 'that was it', 'Stop', 'exit', 'Quit',
            'Bye', 'byebye'], 'bye' : ['Please come back soon!', 
            'I\'ll miss you friend!', 'See you soon!'], }

In [14]:
def bow_chat():
    print('BOT: Hello, I\'m a new chatbot you can talk to me here: ')
    print('')
    while True:
        question = text_nomalization(input('HUMAN: '))
        if question in text_nomalization(Responses['Quit']):
            print('')
            print('BOT: '+random.choice(Responses['bye']))
            break
        else:
            print('')
            print('BOT: ' + bow(question))
            print('')
        
bow_chat()

BOT: Hello, I'm a new chatbot you can talk to me here: 

HUMAN: What is your name

BOT: I can help you work smarter instead of harder

HUMAN: can i call you lexi

BOT: I'm not programmed for that exact question. Try asking another way?

HUMAN: i want to speak with you

BOT: I'm always here to lend an ear.

HUMAN: i will be back in a moment

BOT: You were missed.

HUMAN: bye

BOT: I'll miss you friend!


NOW LET'S TRY THE SAME CHAT BOT USING Tfidf

In [11]:
def tfidf(text):
    tfidf = TfidfVectorizer()
    vector_tfidf = tfidf.fit_transform(df['lematized_context']).toarray()
    features = tfidf.get_feature_names_out()
    tfidf_vector = pd.DataFrame(vector_tfidf, columns = features)
    lema_text = text_nomalization(text)
    tfidf_text = tfidf.transform([lema_text]).toarray()
    cosine_value = 1-pairwise_distances(vector_tfidf, tfidf_text, metric = 'cosine')
    index_val = cosine_value.argmax()
    return 'BOT: ' + df['Text Response'].loc[index_val]

In [15]:
def tfidf_chat():
    print('BOT: Hello, I\'m a new chatbot you can talk to me here: ')
    print('')
    while True:
        question = text_nomalization(input('HUMAN: '))
        if question in text_nomalization(Responses['Quit']):
            print('')
            print('BOT: '+random.choice(Responses['bye']))
            break
        else:
            print('')
            print(tfidf(question))
            print('')
        
tfidf_chat()

BOT: Hello, I'm a new chatbot you can talk to me here: 

HUMAN: what is your name

BOT: I don't have a name yet, I'll be happy if you can choose one for me 

HUMAN: can i call you lexi

BOT: that's a rely cool name I like it.

HUMAN: i want to speak with you

BOT: I'm always here to lend an ear.

HUMAN: i will be back in a moment

BOT: Till next time.

HUMAN: byebye

BOT: See you soon!


Now lest try to add some sentiment analysis to make things more interesting

In [12]:
SentimentResponses = {'negative':['You seem to be in a bad mood. ','That sounds weired coming from you. ',
                    'Whatever you say, hefe. ', 'Awww...are you having a bad day? ', 'Woof, you totally hate. ',
                     'How is that supposed to make me feel? ',
                    'Not too many people like you, do they? ',], 'positive' :['You are clearly feeling positive today. ',
                    'That\'s a very accepting mood. ',
                    'Your positive mood is contagious! ']}

In [13]:
#Incoparating sentiment analysis for the input
def sentiment_bow(text):
    sia = SentimentIntensityAnalyzer()
    text_sentiment = sia.polarity_scores(text)['compound']
    if text_sentiment > 0:
        print('BOT: '+ random.choice(SentimentResponses['positive']) + bow(text))
    elif text_sentiment < 0:
        print('BOT: '+ random.choice(SentimentResponses['negative']) + bow(text))
    else:
        print('BOT: '+bow(text))

In [16]:
def sentiment_bow_chat():
    print("BOT: Hello, I'm a new chatbot you can talk to me here: ")
    print('')
    while True:
        Q= text_nomalization(input('HUMAN: '))
        if Q in text_nomalization(Responses['Quit']):
            print('')
            print('BOT: '+random.choice(Responses['bye']))
            break
        else:
            print('')
            sentiment_bow(Q)
            print('')
            
sentiment_bow_chat()

BOT: Hello, I'm a new chatbot you can talk to me here: 

HUMAN: youre funny i like you

BOT: Your positive mood is contagious! I like you too.

HUMAN: i am happy

BOT: Your positive mood is contagious! Excellent! That's what I like to see.

HUMAN: i am good

BOT: That's a very accepting mood. Great! Glad to hear it.

HUMAN: i dont want to talk to you

BOT: Awww...are you having a bad day? Sure thing. I'll be here if you change your mind.

HUMAN: you are boring

BOT: How is that supposed to make me feel? I'm sorry you think so. We can talk about something more interesting.

HUMAN: you are annoying

BOT: Woof, you totally hate. Sorry to come across that way.

HUMAN: why are you here

BOT: I like being here, I want to make some friends

HUMAN: it's bed time

BOT: Good night. Talk to you later.

HUMAN: stop

BOT: Please come back soon!


CONCLUSION:
  Both methods (BOW and Tfidf) depend greatly on the size and quality of the dialog dataset being used. however tfidf method seem to be somehow more precise than the BOW method.